# Imports

In [35]:
# Imports
import numpy as np
import pandas as pd
import pyspark
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import contractions
import math
from itertools import chain
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import bigrams
from nltk.stem import WordNetLemmatizer

import re
import string
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

from google.cloud import bigquery

from sklearn.feature_extraction.text import TfidfVectorizer 

import pysheets

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jupyter/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [36]:
client = bigquery.Client(location="US")
print("Client creating using default project: {}".format(client.project))

client = bigquery.Client(location="US", project="helio-staging")

Client creating using default project: helio-staging


# Constants + Functions

In [37]:
GROUP_CUSTOM_STOPWORDS = [
    "bike", "nbsp", "crn","sample", "bikes", "great", "good", "love","like", "recommend","smells","collected","part","review","promotion","razor",
    "shave","shaving","blade","camille","cantu","daughter","today","kccc","jessie","miss","fabric","pillow","softener","or",    "tiktok","OR",
     'Proov', 'proov','Natalist', 'natalist','Stix', 'stix','Clear Blue', 'blue','Modern Fertility', 'modern', 'free', 'app', 'mf', 'android', 'pinchme',
    'Pregmate', 'pregmate', 'received','First Response', 'test', 'ovulation', 'none', 'see', 'store', 'day', 'month', 'amy', 'face', 'far', 'scan',
    'it‚äôs','don‚äôt', 'using_minoxidil', '_oz','i‚äôm','moreread','using','stating',
    'doesn‚äôt', 'classic','order','ordering', 'best', 'hard', 'seltzer', 'claw','never','got','ordered','white','product','high-noon','bottle','girl',
    'read_lessread','read_moreread','lessread_le','le_stating','moreread_stating','scalp detox','le_serum',
    'moreread_serum','i’ve_stem','hair_read','don’t_know','i’ve_bought', 'liquid_kinda', "i'ḿ"'use_serum','le_mus','read_serum','ingredient_us','oz_extreme',
    'cell_serum','cell','stem','le','read','lessread','moreread','more','multi-colored—from_brown','findin_read','strengthening_hair','hair_oil','care_oil',
    'hair','oil_hair','extreme_hair','hair_care','extreme','oz','_strengthening','used twice', 'oil_oil', 'customer',    'nan', 'read_moreread', 'noreferrer',
    'noreferrer_farmer','farmer_dog/a','noreferrer_ollie/a','rel','noopener','blank_rel',
    'rel_nofollow','blank','nofollow','target_blank','noopener_noreferrer','nofollower_noopener','nofollow_noopener',"partake","partake_cooky", "crunchy_cooky","cooky_vegan","madegood","apple_jack"
]

added_stopwords_li = [
    "it’s",    "'d",    "'s",
    "n't",    "'m",
    "i've",    "it's",    "'ve",
    "'re",    "'ll",    "``",    "''",    "...",    "--", "https",
    "voxbox",    "influenster",    "cracker",    "or",    'it‚äôs',
    'don‚äôt',    'i‚äôm',    'doesn‚äôt',    "get", "also",    "even",    "since", "amy"
]

REV_COLS = ['source_name',
    'normalized_url',
    'review_source_id',
    'reviewer_source_id',
    'brand_name',
    'product_name',
    'product_source_id',
    'review_date',
    'review_rating',
    'review_content'
]

product_categories = ['Accessories',
'Alcoholic Beverages',
'Apparel & Footwear',
'Food',
'Food Establishments',
'Health & Beauty Establishments',
'Household Consumables',
'Household Durables',
'Other',
'Pet Products',
'Non-Alcoholic Beverages',
'Personal Care & Beauty',
'Retailers',
'Sports & Outdoors'
]

drop_words = ['tablespoon_jot',
              'micro_dose',
              'javy_make',
              'coffe_make',
             ]

MIN_TERM_PCT = 0

MAX_TERM_PCT = 100

TERM_PERCENT_HEAD = 1

TOP_N_WORDS = 8

np.random.seed(42) # set seed for models for reproducibility

stopwords_li = stopwords.words('english')
punkts_li = list(string.punctuation)

REVIEWS_PER_BRAND_TO_ANALYZE = 10_000

In [38]:
def tokenize(s, strip_str="=-_/\+.:,'* 1234567890—"):
    custom_words_li = GROUP_CUSTOM_STOPWORDS + added_stopwords_li+stopwords_li + punkts_li + [""]
    return [w.lower().strip(strip_str) for w in word_tokenize(str(s)) if w.lower().strip(strip_str) not in custom_words_li]

def lemmatize(tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in tokens]

def unique_list(li):
    deduped_li = list(set(li))
    return deduped_li

def bigram(tokens):
    bi_tup = list(bigrams(tokens))
    bi_li = ['_'.join(tup) for tup in bi_tup]
    return bi_li


# Load Data

In [39]:
GROUP_NAME = "Coffee Concentrates"
# GROUP_NAME = 'Meat Substitutes'


# When using Streamlit, leave list empty and un-comment cells below

#Cereals
NORMALIZED_URL_LI = [
    "jot.co",
    "javycoffee.com"

    
]

PRIMARY_CATEGORIES = [

]                      


BRAND_NAME_LI = [
    "Jot",
    "Javy"
    
]


brand_name_di = dict(zip(NORMALIZED_URL_LI, BRAND_NAME_LI))

external_revs_list = []



    
PROD_CAT_EXCLUDE_LI = ['Pet Products']
    
PRODUCT_NAME_NOT_CONTAINS_LI = ['toothpaste']


In [40]:
job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("url", "STRING", NORMALIZED_URL_LI),
        bigquery.ArrayQueryParameter("cols", "STRING", REV_COLS),
        bigquery.ArrayQueryParameter("brands", "STRING", BRAND_NAME_LI)
    ]
)

In [41]:
job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ArrayQueryParameter("url", "STRING", NORMALIZED_URL_LI)
        ]
)
max_date_query = """
    SELECT max(execution_date)
    FROM `helio-staging.online_reviews.online_reviews__deduped_review__1_0`
    where execution_date > DATE_ADD(current_date(), INTERVAL -10 DAY)
    and normalized_url IS NOT NULL
    """
query_job = client.query(
    max_date_query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
    job_config=job_config
)  # API request - starts the query

max_date = query_job.to_dataframe()['f0_'].astype('str')[0]


In [42]:
max_date

'2022-01-26'

In [43]:
job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("url", "STRING", NORMALIZED_URL_LI),
        bigquery.ScalarQueryParameter("max_date", "STRING", max_date)
    ]
)
review_query = """
    SELECT source_name,
    normalized_url,
    review_source_id,
    reviewer_source_id,
    product_name,
    product_source_id,
    review_date,
    review_rating,
    review_content
    FROM `helio-staging.online_reviews.online_reviews__deduped_review__1_0`
    WHERE (normalized_url IN UNNEST (@url)
    OR product_name LIKE '%Jot Concentrated%'
    )
    AND execution_date = @max_date
    """
query_job = client.query(
    review_query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
    job_config=job_config
)  # API request - starts the query

reviews = query_job.to_dataframe()
reviews = reviews.loc[reviews.astype(str).drop_duplicates().index]


In [47]:
reviews.shape
reviews = reviews[reviews["normalized_url"]!="jot.co"]
reviews["normalized_url"] = reviews["normalized_url"].replace("acuitybrands.com","jot.co")
reviews

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,source_name,normalized_url,review_source_id,reviewer_source_id,product_name,product_source_id,review_date,review_rating,review_content
200,amazon,javycoffee.com,R2SV9P1K3IWWH9,amzn1.account.AGPWAADHPMXOGSZNS5AIUEPR4IYQ,Javy Coffee Microdose 30X Liquid Coffee Concen...,B08SJ4HVSY,2021-03-08 00:00:00,5.0,"THIS IS IT! I've been seeing ads for ""concent..."
201,amazon,javycoffee.com,R1Q8VLQ79S37SE,amzn1.account.AGFQIEPCHM7OZQVKSJCXVE3SCPGA,Javy Coffee Microdose 30X Liquid Coffee Concen...,B08SJ4HVSY,2021-03-17 00:00:00,1.0,The bottle wasn't sealed correctly. The foil ...
202,amazon,javycoffee.com,R2D2CN8LEMW5W2,amzn1.account.AFGCEHXJAOM4CMSA5AYBZHPZYYGA,Javy Coffee Microdose 30X Liquid Coffee Concen...,B08SJ4HVSY,2021-03-03 00:00:00,5.0,This stuff is so good and really convenient!
203,amazon,javycoffee.com,R2T3X0NUZMXLO2,amzn1.account.AHESZFF4KDLDJYT3FY6XRGOUWPTA,Javy Coffee Microdose 30X Liquid Coffee Concen...,B08SJ4HVSY,2021-03-15 00:00:00,4.0,Somewhat expensive. Convenient.
204,amazon,javycoffee.com,RR26IKZ06FX4M,amzn1.account.AE3XIZ2JG6HNO4OMNYTZVNUCGXKA,Javy Coffee Microdose 30X Liquid Coffee Concen...,B08SJ4HVSY,2021-03-24 00:00:00,5.0,I really like the convenience of it!!!!
...,...,...,...,...,...,...,...,...,...
438,amazon,javycoffee.com,R1K0DRQUE7H4RD,amzn1.account.AE3JLPTVSOPBQPHSMOSJYAOL4YBQ,Javy Coffee Microdose 30X Liquid Coffee Concen...,B08SJ4HVSY,2021-03-24 00:00:00,5.0,"Love the taste, love the compactness of the b..."
439,amazon,javycoffee.com,R3P8RQ3A87P9MB,amzn1.account.AEWUZW3WW4HC3JEFLIFGCL2KZQUA,Javy Coffee Microdose 30X Liquid Coffee Concen...,B08SJ4HVSY,2021-03-18 00:00:00,5.0,"Convenient, great taste, love it!!"
440,amazon,javycoffee.com,R3FTL5YWTT0RDS,amzn1.account.AERLY2PT4YHQZ47D6B3AHKVGLXVQ,Javy Coffee Microdose 30X Liquid Coffee Concen...,B08SJ4HVSY,2021-03-11 00:00:00,5.0,Great flavor and will save so much money not ...
441,amazon,javycoffee.com,R1BSPIG8LQIMAA,amzn1.account.AHFLYVRZKR2EG6BD74B3RJPGXHVQ,Javy Coffee Microdose 30X Liquid Coffee Concen...,B08SJ4HVSY,2021-03-04 00:00:00,1.0,This stuff is terrible. Not in an offensive w...


In [48]:
# Product Category Data

query = """
    SELECT product_source_id, parent_category, primary_category
    FROM `helio-staging.taxonomy.product_taxonomy__1_0`
    WHERE normalized_url IN UNNEST (@url)

    """
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
    job_config=job_config
)  # API request - starts the query

prod_cat = query_job.to_dataframe()

In [49]:
# All primary categories

query = """
    SELECT distinct parent_category
    FROM `helio-staging.taxonomy.product_taxonomy__1_0`
    """
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
    job_config=job_config
)  # API request - starts the query

cat_list = query_job.to_dataframe()

In [50]:
# Join reviews with product categories

reviews = pd.merge(
    reviews, 
    prod_cat, 
    on='product_source_id', how='left').drop_duplicates(subset=['source_name', 'review_source_id'])

# Filter out excluded categories, excluded product name terms
PROD_CAT_EXCLUDE_LI = ['Pet Products']

for term in PROD_CAT_EXCLUDE_LI:
    reviews = reviews[(~reviews['parent_category'].fillna('').str.contains(term))]

    
PRODUCT_NAME_NOT_CONTAINS_LI = ['Purina', 'Krill']
for term in PRODUCT_NAME_NOT_CONTAINS_LI:
    reviews = reviews[~(reviews['product_name'].str.lower().fillna('').str.contains(term.lower()))]

In [51]:
# Read external CSVs

if external_revs_list != []:
    cleaned_external_review_list = []
    for external_revs_csv in external_revs_list:  
        external_revs_csv['review_rating']=pd.to_numeric(external_revs_csv['review_rating'], downcast='float')
        external_revs_csv['source_name'] = 'DTC'
        # Checking for static columns, to add blanksto data frame
        # Then checking for incrementing columns, which we'll set as the index, assuming they are unique reviewers and reviews
        static_columns = ['product_name', 'product_source_id']
        incrementing_columns = ['review_source_id','reviewer_source_id']

        for column_name in static_columns:
            if column_name not in external_revs_csv:
                external_revs_csv[column_name] = ''
            else:
                pass
        for column_name in incrementing_columns:
            if column_name not in external_revs_csv:
                external_revs_csv[column_name]=external_revs_csv.index
            else:
                pass

        # selecting the same columns as internal reviews CSV
        external_revs_csv = external_revs_csv[brand_revs.columns]
        cleaned_external_review_list.append(external_revs_csv)

    external_revs_csv = pd.concat(cleaned_external_review_list)

In [52]:
# Join external CSVs and clean reviews

if external_revs_list != []:
    reviews = reviews.union(external_revs_sdf)

reviews['review_content'] = reviews['review_content'].str.replace('/[^a-zA-Z0-9]/g', '', regex=True);

# removing hypertext from reviews
reviews['review_content'] = reviews['review_content'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

# removing contractions from reviews
reviews['review_content'] = reviews['review_content'].apply(lambda x: [contractions.fix(word) for word in x.split()])

In [53]:
# Get most recent n reviews

reviews = (reviews
           .sort_values('review_date',ascending = False)
           .groupby('normalized_url')
           .head(REVIEWS_PER_BRAND_TO_ANALYZE)
          )

reviews_count = reviews.groupby('normalized_url').agg({'review_source_id':'count'}).reset_index().rename(columns={"review_source_id": "rev_count"})

reviews = pd.merge(reviews_count, reviews, on=['normalized_url'])

#reviews = reviews[reviews['primary_category'].str.contains('|'.join(PRIMARY_CATEGORIES), na=False)]

In [54]:
#reviews[reviews["normalized_url"]=="jot.co"]


In [55]:
tokenized_reviews = reviews.copy()
tokenized_reviews['tokenized_review'] = tokenized_reviews['review_content'].apply(lambda x: lemmatize(tokenize(x)))
tokenized_reviews = tokenized_reviews.dropna(subset=['tokenized_review'])

In [56]:
unibi_reviews = tokenized_reviews.copy()

unibi_reviews['unibi'] = unibi_reviews['tokenized_review'].apply(lambda x: unique_list(bigram(x)+x))
unibi_reviews = unibi_reviews.loc[unibi_reviews.astype(str).drop_duplicates().index]

unibi_words = unibi_reviews.copy()
unibi_words['unibi_word'] = unibi_words['unibi']
unibi_words = unibi_words.explode('unibi_word')
unibi_words= unibi_words[~unibi_words["unibi_word"].isin(drop_words)]
#unibi_words=unibi_words[unibi_words["parent_category"]=="Food"]

In [57]:
unibi_words

,normalized_url,rev_count,source_name,review_source_id,reviewer_source_id,product_name,product_source_id,review_date,review_rating,review_content,parent_category,primary_category,tokenized_review,unibi,unibi_word
0,javycoffee.com,243,amazon,R2UQVJZ8O0KDXE,amzn1.account.AGMEQMSQG7CXIVRLNMDOURR33CAQ,Javy Coffee Microdose 30X Liquid Coffee Concen...,B08SJ4HVSY,2021-03-26 00:00:00,2.0,"[This, coffee, tastes, horrible., Bitter, and,...",Non-Alcoholic Beverages,Coffee,"[coffee, taste, horrible, bitter, burnt]","[horrible, coffee, horrible_bitter, bitter_bur...",horrible
0,javycoffee.com,243,amazon,R2UQVJZ8O0KDXE,amzn1.account.AGMEQMSQG7CXIVRLNMDOURR33CAQ,Javy Coffee Microdose 30X Liquid Coffee Concen...,B08SJ4HVSY,2021-03-26 00:00:00,2.0,"[This, coffee, tastes, horrible., Bitter, and,...",Non-Alcoholic Beverages,Coffee,"[coffee, taste, horrible, bitter, burnt]","[horrible, coffee, horrible_bitter, bitter_bur...",coffee
0,javycoffee.com,243,amazon,R2UQVJZ8O0KDXE,amzn1.account.AGMEQMSQG7CXIVRLNMDOURR33CAQ,Javy Coffee Microdose 30X Liquid Coffee Concen...,B08SJ4HVSY,2021-03-26 00:00:00,2.0,"[This, coffee, tastes, horrible., Bitter, and,...",Non-Alcoholic Beverages,Coffee,"[coffee, taste, horrible, bitter, burnt]","[horrible, coffee, horrible_bitter, bitter_bur...",horrible_bitter
0,javycoffee.com,243,amazon,R2UQVJZ8O0KDXE,amzn1.account.AGMEQMSQG7CXIVRLNMDOURR33CAQ,Javy Coffee Microdose 30X Liquid Coffee Concen...,B08SJ4HVSY,2021-03-26 00:00:00,2.0,"[This, coffee, tastes, horrible., Bitter, and,...",Non-Alcoholic Beverages,Coffee,"[coffee, taste, horrible, bitter, burnt]","[horrible, coffee, horrible_bitter, bitter_bur...",bitter_burnt
0,javycoffee.com,243,amazon,R2UQVJZ8O0KDXE,amzn1.account.AGMEQMSQG7CXIVRLNMDOURR33CAQ,Javy Coffee Microdose 30X Liquid Coffee Concen...,B08SJ4HVSY,2021-03-26 00:00:00,2.0,"[This, coffee, tastes, horrible., Bitter, and,...",Non-Alcoholic Beverages,Coffee,"[coffee, taste, horrible, bitter, burnt]","[horrible, coffee, horrible_bitter, bitter_bur...",coffee_taste
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,javycoffee.com,243,amazon,R4H31SY96NBTP,amzn1.account.AFQPSWWJP2R2PXNC5RCINEMWRIBQ,Javy Coffee Microdose 30X Liquid Coffee Concen...,B08SJ4HVSY,2021-01-26 00:00:00,5.0,"[Would, recommend, 100/10!, I, saved, so, much...",Non-Alcoholic Beverages,Coffee,"[would, saved, much, time, morning, hate, morn...","[interact, line, home, time_morning, saved, wa...",happy_make
242,javycoffee.com,243,amazon,R4H31SY96NBTP,amzn1.account.AFQPSWWJP2R2PXNC5RCINEMWRIBQ,Javy Coffee Microdose 30X Liquid Coffee Concen...,B08SJ4HVSY,2021-01-26 00:00:00,5.0,"[Would, recommend, 100/10!, I, saved, so, much...",Non-Alcoholic Beverages,Coffee,"[would, saved, much, time, morning, hate, morn...","[interact, line, home, time_morning, saved, wa...",happy
242,javycoffee.com,243,amazon,R4H31SY96NBTP,amzn1.account.AFQPSWWJP2R2PXNC5RCINEMWRIBQ,Javy Coffee Microdose 30X Liquid Coffee Concen...,B08SJ4HVSY,2021-01-26 00:00:00,5.0,"[Would, recommend, 100/10!, I, saved, so, much...",Non-Alcoholic Beverages,Coffee,"[would, saved, much, time, morning, hate, morn...","[interact, line, home, time_morning, saved, wa...",right_front
242,javycoffee.com,243,amazon,R4H31SY96NBTP,amzn1.account.AFQPSWWJP2R2PXNC5RCINEMWRIBQ,Javy Coffee Microdose 30X Liquid Coffee Concen...,B08SJ4HVSY,2021-01-26 00:00:00,5.0,"[Would, recommend, 100/10!, I, saved, so, much...",Non-Alcoholic Beverages,Coffee,"[would, saved, much, time, morning, hate, morn...","[interact, line, home, time_morning, saved, wa...",home_waiting


In [58]:
grouped_sdf = pd.DataFrame(unibi_reviews.groupby('normalized_url').unibi.apply(sum)).reset_index()

In [59]:
pd.set_option('max_colwidth', 50)

#tokenized_reviews[(tokenized_reviews['review_source_id']=='232203656')]

grouped_sdf['unibi_str'] = [','.join(map(str, l)) for l in grouped_sdf['unibi']]

In [60]:
grouped_sdf

,normalized_url,unibi,unibi_str
0,javycoffee.com,"[horrible, coffee, horrible_bitter, bitter_bur...","horrible,coffee,horrible_bitter,bitter_burnt,c..."


In [61]:
frames = []

for i in range(len(grouped_sdf)):    
# settings that you use for count vectorizer will go here 
    tfidf_vectorizer=TfidfVectorizer(
        norm=None,
        sublinear_tf=False,
        use_idf=True,
        lowercase=False,
        stop_words='english',
        smooth_idf=True,
        max_df = len(grouped_sdf['normalized_url'])-1
    ) 

    # just send in all your docs here 
    tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(grouped_sdf['unibi_str'])

    # get the first vector out (for the first document) 
    first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[i]

    # place tf-idf values in a pandas data frame 
    df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
    joined_df = unibi_words[(unibi_words['normalized_url']==grouped_sdf.reset_index()['normalized_url'][i])].join(df, on='unibi_word', how='inner', lsuffix='_left', rsuffix='_right')
    frames.append(joined_df)

df = pd.concat(frames)
#reviews = unibi_reviews[(unibi_reviews['normalized_url'] == url)]
#joined_df = unibi_reviews.join(df, on='unibi_word', how='inner', lsuffix='_left', rsuffix='_right')
sorted_df = df[['normalized_url', 'unibi_word', 'tfidf']].sort_values(by=['normalized_url', "tfidf"],ascending=False).drop_duplicates()


ValueError: max_df corresponds to < documents than min_df

In [ ]:
tf_idf_output = sorted_df.groupby('normalized_url').head(TOP_N_WORDS)

In [ ]:
unibi_reviews_grouped = unibi_words.groupby(['normalized_url', 'unibi_word']).agg({'review_rating':'mean', 'review_source_id':'count'})

unibi_reviews_grouped = unibi_reviews_grouped.rename(columns={"review_rating": "avg_rating", "review_source_id": "review_count"})

In [ ]:
avg_rating_tfidf = pd.merge(unibi_reviews_grouped, tf_idf_output, on=['normalized_url', 'unibi_word'], how='inner').sort_values(by=['normalized_url', "tfidf"],ascending=False)[['normalized_url','unibi_word', 'tfidf', 'avg_rating']]

In [ ]:
kpc = pd.read_csv("coffee kpc.csv")

In [ ]:
kpc['attribute_prevalence'] = kpc['attribute_prevalence']/kpc['attribute_prevalence'].sum()
kpc

In [ ]:
#map aspects to words
kpc_di = dict(zip(kpc["seed_words"],kpc["attribute_name"]))
seed_words = kpc["seed_words"].tolist()
attributes = kpc["attribute_name"].tolist()

def aspect_map (x):
    for i in seed_words:
        if x in i.split(","):
            return kpc_di[i]


unibi_words["aspect"] = unibi_words["unibi_word"].map(aspect_map)
unibi_words["brand"] = unibi_words["normalized_url"].map(brand_name_di)

In [ ]:
unibi_words

In [ ]:
#calculate aspect rating
#unibi_words.groupby(["brand", "aspect"])["review_rating"].mean()
brand_aspects = unibi_words[unibi_words["aspect"].isin(attributes)].groupby(["brand", "aspect"])["review_rating"].mean().reset_index()
rating_data = brand_aspects

In [ ]:
rating_data["asp_prev"] = rating_data["aspect"].map(dict(zip(kpc["kpc_name"],kpc["attribute_prevalence"])))
rating_data = rating_data.sort_values(by="asp_prev", ascending=False)
rating_data= rating_data.sort_values(by="brand")

In [ ]:

fig = make_subplots(specs=[[{"secondary_y": True}]])

bar_data =  px.bar(
        kpc,
        x='attribute_name',
        y='attribute_prevalence',
        barmode='group',
    )

line_data = px.scatter(
        rating_data,
        x='aspect',
        y="review_rating",
        color='brand',
        color_discrete_sequence=["blue", "goldenrod", "magenta"],
    )

fig.update_xaxes(categoryorder='array', categoryarray= ['Flavor', 'Value for money','Packaging','Freshness','Giftable','Easy to use'])

for data in line_data.data:
    fig.add_trace(data.update(mode='markers+lines'), secondary_y=True)
    

for data in bar_data.data:
    fig.add_trace(data)
    
fig.update_traces(marker=dict(color="lightgray"),selector=dict(type="bar"))

fig.update_layout(
    title=f"KPC for Select {GROUP_NAME} Brands",
    xaxis_title="Aspect",
    yaxis_title="Overall Normalized Percent Weight",
)



fig.update_yaxes(
    range=[1,5],
    title="Brand Stars Rating (out of 5)",
    secondary_y=True
)
fig.update_yaxes(
    autorange = True,
    tickformat='.0%',
    secondary_y=False
)

fig.update_layout(
    width=1200,
    height=800,
    plot_bgcolor="white",
    title={"x": 0.5},
    coloraxis={"autocolorscale": True},
    paper_bgcolor ="rgba(0,0,0,0)",
  
)

fig.show()